In [304]:
from bokeh.models.tools import (
    BoxZoomTool, BoxZoomTool, HelpTool, LassoSelectTool, PanTool,
    PreviewSaveTool, ResetTool, ResizeTool, WheelZoomTool
)

# Import other general libs

In [280]:
import json
import codecs

## Miss data

In [285]:
MISS_PATH_TO_DOCUMENT_SVD = './data_for_visual/miss_document_svd_obj.json'
miss_document_svd_result = json.loads(codecs.open(MISS_PATH_TO_DOCUMENT_SVD, 'r', 'utf-8').read())

In [286]:
MISS_PATH_TO_DOCUMENT_tsne = './data_for_visual/miss_document_tsne_obj.json'
miss_document_tsne_result = json.loads(codecs.open(MISS_PATH_TO_DOCUMENT_tsne, 'r', 'utf-8').read())

In [287]:
MISS_PATH_TO_PICS_tsne = './data_for_visual/miss_pics_tsne_obj.json'
miss_pics_tsne_result = json.loads(codecs.open(MISS_PATH_TO_PICS_tsne, 'r', 'utf-8').read())

## Mr

In [288]:
MR_PATH_TO_DOCUMENT_SVD = './data_for_visual/mr_document_svd_obj.json'
mr_document_svd_result = json.loads(codecs.open(MR_PATH_TO_DOCUMENT_SVD, 'r', 'utf-8').read())

In [289]:
MR_PATH_TO_DOCUMENT_tsne = './data_for_visual/mr_document_tsne_obj.json'
mr_document_tsne_result = json.loads(codecs.open(MR_PATH_TO_DOCUMENT_tsne, 'r', 'utf-8').read())

In [290]:
MR_PATH_TO_PICS_tsne = './data_for_visual/mr_pics_tsne_obj.json'
mr_pics_tsne_result = json.loads(codecs.open(MR_PATH_TO_PICS_tsne, 'r', 'utf-8').read())

# Functions to make scatter graph

In [317]:
def format_item_values(document_svd_result, image_root_url):

    # This should be function
    XY_list = [member_obj['position_vector'] for member_obj in document_svd_result.values()]
    universities = [member_obj['university'] for member_obj in document_svd_result.values()]
    major = [member_obj['major'] for member_obj in document_svd_result.values()]
    height = [member_obj['height'] for member_obj in document_svd_result.values()]
    age = [member_obj['age'] for member_obj in document_svd_result.values()]
    X = [xy[0] for xy in XY_list]
    Y = [xy[1] for xy in XY_list]
    labels = [member_obj['member_name'] for member_obj in document_svd_result.values()]
    #images = [str(member_obj['photo_url'].replace(u'//img', u'/img')) for member_obj in document_svd_result.values()]
    images = ['{}{}'.format(image_root_url, name_rubi) for name_rubi in document_svd_result.keys()]
    blog_links = [member_obj['blog_url'] for member_obj in document_svd_result.values()]
    profile_links = [member_obj['profile_url'] for member_obj in document_svd_result.values()]
    
    return {
        'X': X,
        'Y': Y,
        'universities': universities,
        'major': major,
        'height': height,
        'age': age,
        'labels': labels,
        'images': images,
        'blog_links': blog_links,
        'profile_links': profile_links
    }

In [318]:
def generate_graph(items_for_graph, graph_title_name, graph_html_name):
    # 
    from bokeh.io import output_file, show, vform, vplot
    from bokeh.plotting import figure, output_file, show, ColumnDataSource
    from bokeh.models import HoverTool, OpenURL, TapTool    
    # Import bokeh, sub modules for making scallter graph with tooltip
    from bokeh.models.widgets import DataTable, DateFormatter, TableColumn    
    from bokeh.models import ColumnDataSource, OpenURL, TapTool
    from bokeh.plotting import figure, output_file, show    
    
    output_file(graph_html_name)
    GRAPH_HEIGHT = 1000
    GRAPH_WIDTH = 800
    TOOLS = [WheelZoomTool(), PanTool(), ResetTool(), TapTool()]
    #TOOLS = []

    source = ColumnDataSource(
            data=dict(
            x=items_for_graph['X'],
            y=items_for_graph['Y'],
            desc=items_for_graph['labels'],
            imgs = items_for_graph['images'],
            univ = items_for_graph['universities'],
            major = items_for_graph['major'],
            height = items_for_graph['height'],
            age = items_for_graph['age'],
            blog = items_for_graph['blog_links'],
            profile= items_for_graph['profile_links'],
            )
        )
    

    hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <img
                    src="@imgs" height="100" alt="@imgs" width="100"
                    style="float: left; margin: 0px 15px 40px 0px;"
                    border="2"
                >
                <br>
                </img>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;"><nobr>@desc さん</nobr></span>
                <span style="font-size: 12px; color: #966;">[$index]</span>
            </div>
            <div>
                <span style="font-size: 10px;">University: @univ</span><br>
                <span style="font-size: 10px;">Major: @major</span><br>
                <span style="font-size: 10px;">age: @age</span><br>
                <span style="font-size: 10px;">height: @height</span><br>
            </div>            
            <div>
                <span style="font-size: 12px;">Position</span>
                <span style="font-size: 10px; color: #696;">($x, $y)</span>
            </div>
        </div>
        """
        )


    s1 = figure(plot_width=GRAPH_WIDTH, plot_height=GRAPH_HEIGHT, tools=[hover]+TOOLS,
               title=graph_title_name)
    s1.circle('x', 'y', size=10, source=source)
    url = "@profile"
    taptool = s1.select(type=TapTool)
    taptool.callback = OpenURL(url=url)
    
    columns = [
        TableColumn(field="desc", title="Name"),
        TableColumn(field="univ", title="University"),
        TableColumn(field="major", title="Major"),
        TableColumn(field="age", title="Age"), 
        TableColumn(field="height", title="Height")
    ]
    data_table = DataTable(source=source, columns=columns, width=1000, height=500)
    s2 = vform(data_table)
    
    p = vplot(s1, s2)

    show(p)

## generate scatter graph by document and svd

## make girls' scatter graph

In [311]:
items_for_graph = format_item_values(miss_document_svd_result, image_root_url = IMAGE_ROOT_URL)
IMAGE_ROOT_URL = 'http://kensuke-mi.pussycat.jp/univ_contest_project_pics/miss_pictures/'
generate_graph(items_for_graph, 
               graph_title_name='girls similarity based on document svd',
               graph_html_name="girls_document_svd_scatter.html"

In [312]:
IMAGE_ROOT_URL = 'http://kensuke-mi.pussycat.jp/univ_contest_project_pics/miss_pictures/'
items_for_graph_pics_tsne = format_item_values(miss_pics_tsne_result, image_root_url = IMAGE_ROOT_URL)
generate_graph(items_for_graph_pics_tsne, 
               graph_title_name='girls similarity based on pictures tsne', 
               graph_html_name="girls_pics_tsne_scatter.html")

## make boys' scatter graph

In [319]:
items_for_graph = format_item_values(mr_document_svd_result, image_root_url = IMAGE_ROOT_URL)
IMAGE_ROOT_URL = 'http://kensuke-mi.pussycat.jp/univ_contest_project_pics/mr_pictures/'
generate_graph(items_for_graph, 
               graph_title_name='boys similarity based on document svd',
               graph_html_name="boys_document_svd_scatter.html")

In [320]:
IMAGE_ROOT_URL = 'http://kensuke-mi.pussycat.jp/univ_contest_project_pics/mr_pictures/'
items_for_graph_pics_tsne = format_item_values(mr_pics_tsne_result, image_root_url = IMAGE_ROOT_URL)
generate_graph(items_for_graph_pics_tsne, 
               graph_title_name='boys similarity based on pictures tsne', 
               graph_html_name="boys_pics_tsne_scatter.html")